In [4]:
from pprint import pprint

### 메타데이터 확인

In [2]:
from datasets import load_dataset_builder


builder = load_dataset_builder("huggan/pokemon")

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Repo card metadata block was not found. Setting CardData to empty.


In [6]:
print(type(builder))

<class 'datasets.packaged_modules.parquet.parquet.ParquetPokemon'>


In [5]:
pprint(builder.info)

DatasetInfo(description='',
            citation='',
            homepage='',
            license='',
            features={'image': Image(mode=None, decode=True, id=None)},
            post_processed=None,
            supervised_keys=None,
            builder_name='parquet',
            dataset_name='pokemon',
            config_name='default',
            version=0.0.0,
            splits={'train': SplitInfo(name='train',
                                       num_bytes=141969739,
                                       num_examples=7357,
                                       shard_lengths=None,
                                       dataset_name=None)},
            download_checksums=None,
            download_size=130657589,
            post_processing_size=None,
            dataset_size=141969739,
            size_in_bytes=192036232.655)


In [7]:
print("features :", builder.info.features)

features : {'image': Image(mode=None, decode=True, id=None)}


In [11]:
print("splits   :", builder.info.splits)

splits   : {'train': SplitInfo(name='train', num_bytes=141969739, num_examples=7357, shard_lengths=None, dataset_name=None)}


In [12]:
print("total MB :", round(builder.info.dataset_size / 1024**2, 2))

total MB : 135.39


In [6]:
import torch

In [8]:
torch.linspace(100, 0, 10, dtype=torch.float)

tensor([100.0000,  88.8889,  77.7778,  66.6667,  55.5556,  44.4444,  33.3333,
         22.2222,  11.1111,   0.0000])

In [1]:
from datasets import load_dataset

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
instruction = 'Convert the equation images to LaTeX equations.'
def convert(sample):
    conversation=[
        {
            'role' : 'user',
            'content' : [
                {'type': 'text', 'text': instruction},
                {'type': 'image', 'image': sample['image']}
            ]
        },
        {
            'role' : 'assistant',
            'content' : [
                {'type': 'text', 'text': sample['text']}
            ]
        }
    ]

    return {'message': conversation}

In [3]:
def process_vision_info(messages):
    image_inputs = []
    for msg in messages:
        content = mes.get("content", [])
        if not isinstance(content, list):
            content = [content]
        
        for element in content:
            if isinstance(element, dict) and ("image" in element or element.get("type") == "image"):
                image = element["image"]
                image_inputs.append(image.convert("RGB"))

    return image_inputs

In [4]:
	
# Load dataset from the hub
dataset_train = load_dataset('unsloth/LaTeX_OCR', split='train[:3000]')

Generating test split: 100%|███████████████████████████████████████████| 7632/7632 [00:00<00:00, 58054.02 examples/s]


In [10]:
from tqdm import tqdm
converted_dataset_train = [
    convert(sample) \
    for sample in tqdm(dataset_train, total=len(dataset_train))
]
print(converted_dataset_train[0])

100%|██████████████████████████████████████████████████████████████████████████| 3000/3000 [00:01<00:00, 2281.25it/s]

{'message': [{'role': 'user', 'content': [{'type': 'text', 'text': 'Convert the equation images to LaTeX equations.'}, {'type': 'image', 'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=160x40 at 0x764CEA7770D0>}]}, {'role': 'assistant', 'content': [{'type': 'text', 'text': '{ \\frac { N } { M } } \\in { \\bf Z } , { \\frac { M } { P } } \\in { \\bf Z } , { \\frac { P } { Q } } \\in { \\bf Z }'}]}]}


In [12]:
import torch
from transformers import AutoProcessor, AutoModelForImageTextToText, BitsAndBytesConfig
 

ImportError: cannot import name 'AutoModelForImageTextToText' from 'transformers' (/opt/conda/lib/python3.10/site-packages/transformers/__init__.py)

In [ ]:
!

In [ ]:
# Create a data collator to encode text and image pairs
def collate_fn(examples):
    texts = []
    images = []
    for example in examples:
        image_inputs = process_vision_info(example["messages"])
        text = processor.apply_chat_template(
            example["messages"], add_generation_prompt=False, tokenize=False
        )
        texts.append(text.strip())
        images.append(image_inputs)
 
    # Tokenize the texts and process the images
    batch = processor(text=texts, images=images, return_tensors="pt", padding=True)
 
    # The labels are the input_ids, and we mask the padding tokens and image tokens in the loss computation
    labels = batch["input_ids"].clone()
 
    # Mask image tokens
    image_token_id = [
        processor.tokenizer.convert_tokens_to_ids(
            processor.tokenizer.special_tokens_map["boi_token"]
        )
    ]
    # Mask tokens for not being used in the loss computation
    labels[labels == processor.tokenizer.pad_token_id] = -100
    labels[labels == image_token_id] = -100
    labels[labels == 262144] = -100
 
    batch["labels"] = labels
    return batch